Analisando mais a fundo para o HT, base de dados diferente porque tem HT (1 tempo)

In [6]:
import pandas as pd

# Carregar o arquivo fornecido
file_path_atualizado = 'C:\\Users\\msdof\\OneDrive\\Imagens\\Ligas\\BASE DE DADOS GRANDES\\futpython.xlsx'
dados = pd.read_excel(file_path_atualizado)

# Filtrar dados específicos para a liga "England Premier League" e a temporada "2022"
dados = dados[(dados['League'] == 'Brazil Serie A') & (dados['Season'].astype(str).str.contains('2022'))]

# Ordenar dados por data e rodada
dados = dados.sort_values(by=['Date', 'Rodada'])

def calcular_probabilidade_variacao(dados, n):
    # Verificar se houve variação nas últimas N rodadas
    condicao_varia = True
    for i in range(1, n+1):
        condicao_varia &= (dados['TotalGoals_HT'] != dados['TotalGoals_HT'].shift(i))

    dados[f'VariaUltimas{n}Rodadas'] = condicao_varia

    # Criar uma nova coluna para a rodada seguinte
    dados['TotalGoals_Proxima_Rodada'] = dados['TotalGoals_HT'].shift(-1)

    # Verificar se há variação na próxima rodada
    dados['VariaProximaRodada'] = dados['TotalGoals_HT'] != dados['TotalGoals_Proxima_Rodada']

    # Filtrar dados conforme a condição estabelecida
    dados_varia = dados[dados[f'VariaUltimas{n}Rodadas']]

    # Contar total de casos onde houve variação nas últimas N rodadas
    total_varia = dados_varia.shape[0]

    # Contar total de casos onde houve variação na próxima rodada após variação nas últimas N
    varia_na_proxima = dados_varia['VariaProximaRodada'].sum()

    # Calcular a probabilidade de variação
    probabilidade_variacao = None
    if total_varia > 0:
        probabilidade_variacao = varia_na_proxima / total_varia

    # Calcular a probabilidade de ter pelo menos 1 gol no HT
    total_jogos_ht_com_gols = dados_varia['TotalGoals_HT'][dados_varia['TotalGoals_HT'] > 0].count()
    probabilidade_gol_ht = total_jogos_ht_com_gols / total_varia if total_varia > 0 else None

    # Calcular a probabilidade de ter pelo menos 2 gols no FT
    total_jogos_ft_com_2_gols = dados_varia['TotalGoals_FT'][dados_varia['TotalGoals_FT'] >= 2].count()
    probabilidade_2_gols_ft = total_jogos_ft_com_2_gols / total_varia if total_varia > 0 else None

    # Calcular a probabilidade de ter no máximo 3 gols no FT
    total_jogos_ft_com_ate_3_gols = dados_varia['TotalGoals_FT'][dados_varia['TotalGoals_FT'] <= 3].count()
    probabilidade_ate_3_gols_ft = total_jogos_ft_com_ate_3_gols / total_varia if total_varia > 0 else None

    return probabilidade_variacao, total_varia, probabilidade_gol_ht, probabilidade_2_gols_ft, probabilidade_ate_3_gols_ft

# Calcular a probabilidade para diferentes valores de N
resultados = {}
for n in range(1, 11):
    probabilidade_variacao, total_jogos_testados, probabilidade_gol_ht, probabilidade_2_gols_ft, probabilidade_ate_3_gols_ft = calcular_probabilidade_variacao(dados, n)
    if probabilidade_variacao is not None:
        resultados[n] = (probabilidade_variacao, total_jogos_testados, probabilidade_gol_ht, probabilidade_2_gols_ft, probabilidade_ate_3_gols_ft)

# Exibir os resultados
for n, (probabilidade_variacao, total_jogos_testados, probabilidade_gol_ht, probabilidade_2_gols_ft, probabilidade_ate_3_gols_ft) in resultados.items():
    print(f"Probabilidade de variação na próxima rodada após variação nas últimas {n} rodadas: {probabilidade_variacao:.2%}")
    print(f"Total de jogos testados para {n} rodadas: {total_jogos_testados}")
    print(f"Probabilidade de ter pelo menos 1 gol no HT: {probabilidade_gol_ht:.2%}")
    print(f"Probabilidade de ter pelo menos 2 gols no FT: {probabilidade_2_gols_ft:.2%}")
    print(f"Probabilidade de ter no máximo 3 gols no FT: {probabilidade_ate_3_gols_ft:.2%}\n")

Probabilidade de variação na próxima rodada após variação nas últimas 1 rodadas: 68.24%
Total de jogos testados para 1 rodadas: 255
Probabilidade de ter pelo menos 1 gol no HT: 69.02%
Probabilidade de ter pelo menos 2 gols no FT: 69.02%
Probabilidade de ter no máximo 3 gols no FT: 74.51%

Probabilidade de variação na próxima rodada após variação nas últimas 2 rodadas: 71.14%
Total de jogos testados para 2 rodadas: 201
Probabilidade de ter pelo menos 1 gol no HT: 70.15%
Probabilidade de ter pelo menos 2 gols no FT: 73.13%
Probabilidade de ter no máximo 3 gols no FT: 71.64%

Probabilidade de variação na próxima rodada após variação nas últimas 3 rodadas: 74.10%
Total de jogos testados para 3 rodadas: 139
Probabilidade de ter pelo menos 1 gol no HT: 74.82%
Probabilidade de ter pelo menos 2 gols no FT: 77.70%
Probabilidade de ter no máximo 3 gols no FT: 65.47%

Probabilidade de variação na próxima rodada após variação nas últimas 4 rodadas: 79.59%
Total de jogos testados para 4 rodadas: 98

Análise para o 2 tempo

In [1]:
import pandas as pd

# Carregar o arquivo fornecido
file_path_atualizado = 'C:\\Users\\msdof\\OneDrive\\Imagens\\Ligas\\BASE DE DADOS GRANDES\\futpython.xlsx'
dados = pd.read_excel(file_path_atualizado)

# Filtrar dados específicos para a liga "Brazil Serie A" e a temporada "2022"
dados = dados[(dados['League'] == 'Brazil Serie A') & (dados['Season'].astype(str).str.contains('2022'))]

# Ordenar dados por data e rodada
dados = dados.sort_values(by=['Date', 'Rodada'])

# Criar coluna para os gols do segundo tempo
dados['TotalGoals_2T'] = dados['TotalGoals_FT'] - dados['TotalGoals_HT']

def calcular_probabilidade_variacao(dados, n):
    # Verificar se houve variação nas últimas N rodadas
    condicao_varia = True
    for i in range(1, n+1):
        condicao_varia &= (dados['TotalGoals_2T'] != dados['TotalGoals_2T'].shift(i))

    dados[f'VariaUltimas{n}Rodadas'] = condicao_varia

    # Criar uma nova coluna para a rodada seguinte
    dados['TotalGoals_Proxima_Rodada'] = dados['TotalGoals_2T'].shift(-1)

    # Verificar se há variação na próxima rodada
    dados['VariaProximaRodada'] = dados['TotalGoals_2T'] != dados['TotalGoals_Proxima_Rodada']

    # Filtrar dados conforme a condição estabelecida
    dados_varia = dados[dados[f'VariaUltimas{n}Rodadas']]

    # Contar total de casos onde houve variação nas últimas N rodadas
    total_varia = dados_varia.shape[0]

    # Contar total de casos onde houve variação na próxima rodada após variação nas últimas N
    varia_na_proxima = dados_varia['VariaProximaRodada'].sum()

    # Calcular a probabilidade de variação
    probabilidade_variacao = None
    if total_varia > 0:
        probabilidade_variacao = varia_na_proxima / total_varia

    # Calcular a probabilidade de ter pelo menos 1 gol no 2T
    total_jogos_2t_com_gols = dados_varia['TotalGoals_2T'][dados_varia['TotalGoals_2T'] > 0].count()
    probabilidade_gol_2t = total_jogos_2t_com_gols / total_varia if total_varia > 0 else None

    # Calcular a probabilidade de ter pelo menos 2 gols no FT
    total_jogos_ft_com_2_gols = dados_varia['TotalGoals_FT'][dados_varia['TotalGoals_FT'] >= 2].count()
    probabilidade_2_gols_ft = total_jogos_ft_com_2_gols / total_varia if total_varia > 0 else None

    # Calcular a probabilidade de ter no máximo 3 gols no FT
    total_jogos_ft_com_ate_3_gols = dados_varia['TotalGoals_FT'][dados_varia['TotalGoals_FT'] <= 3].count()
    probabilidade_ate_3_gols_ft = total_jogos_ft_com_ate_3_gols / total_varia if total_varia > 0 else None

    return probabilidade_variacao, total_varia, probabilidade_gol_2t, probabilidade_2_gols_ft, probabilidade_ate_3_gols_ft

# Calcular a probabilidade para diferentes valores de N
resultados = {}
for n in range(1, 11):
    probabilidade_variacao, total_jogos_testados, probabilidade_gol_2t, probabilidade_2_gols_ft, probabilidade_ate_3_gols_ft = calcular_probabilidade_variacao(dados, n)
    if probabilidade_variacao is not None:
        resultados[n] = (probabilidade_variacao, total_jogos_testados, probabilidade_gol_2t, probabilidade_2_gols_ft, probabilidade_ate_3_gols_ft)

# Exibir os resultados
for n, (probabilidade_variacao, total_jogos_testados, probabilidade_gol_2t, probabilidade_2_gols_ft, probabilidade_ate_3_gols_ft) in resultados.items():
    print(f"Probabilidade de variação na próxima rodada após variação nas últimas {n} rodadas: {probabilidade_variacao:.2%}")
    print(f"Total de jogos testados para {n} rodadas: {total_jogos_testados}")
    print(f"Probabilidade de ter pelo menos 1 gol no 2T: {probabilidade_gol_2t:.2%}")
    print(f"Probabilidade de ter pelo menos 2 gols no FT: {probabilidade_2_gols_ft:.2%}")
    print(f"Probabilidade de ter no máximo 3 gols no FT: {probabilidade_ate_3_gols_ft:.2%}\n")


Probabilidade de variação na próxima rodada após variação nas últimas 1 rodadas: 73.99%
Total de jogos testados para 1 rodadas: 273
Probabilidade de ter pelo menos 1 gol no 2T: 75.82%
Probabilidade de ter pelo menos 2 gols no FT: 68.86%
Probabilidade de ter no máximo 3 gols no FT: 76.92%

Probabilidade de variação na próxima rodada após variação nas últimas 2 rodadas: 76.47%
Total de jogos testados para 2 rodadas: 204
Probabilidade de ter pelo menos 1 gol no 2T: 75.00%
Probabilidade de ter pelo menos 2 gols no FT: 71.08%
Probabilidade de ter no máximo 3 gols no FT: 74.02%

Probabilidade de variação na próxima rodada após variação nas últimas 3 rodadas: 77.63%
Total de jogos testados para 3 rodadas: 152
Probabilidade de ter pelo menos 1 gol no 2T: 73.68%
Probabilidade de ter pelo menos 2 gols no FT: 73.68%
Probabilidade de ter no máximo 3 gols no FT: 69.74%

Probabilidade de variação na próxima rodada após variação nas últimas 4 rodadas: 77.59%
Total de jogos testados para 4 rodadas: 11

In [7]:
import pandas as pd

# Carregar o arquivo fornecido
file_path_atualizado = 'C:\\Users\\msdof\\OneDrive\\Imagens\\Ligas\\BASE DE DADOS GRANDES\\futpython.xlsx'

dados = pd.read_excel(file_path_atualizado)

# Filtrar dados específicos para a liga "England Premier League" e a temporada "2022"
dados = dados[(dados['League'] == 'Brazil Serie A') & (dados['Season'].astype(str).str.contains('2022'))]

#filtro das odds
dados = dados[dados['Odd_A_FT'] < 2]


# Ordenar dados por data e rodada
dados = dados.sort_values(by=['Date', 'Rodada'])

def calcular_probabilidade_variacao(dados, n):
    # Verificar se houve variação nas últimas N rodadas
    condicao_varia = True
    for i in range(1, n+1):
        condicao_varia &= (dados['TotalGoals_HT'] != dados['TotalGoals_HT'].shift(i))

    dados[f'VariaUltimas{n}Rodadas'] = condicao_varia

    # Criar uma nova coluna para a rodada seguinte
    dados['TotalGoals_Proxima_Rodada'] = dados['TotalGoals_HT'].shift(-1)

    # Verificar se há variação na próxima rodada
    dados['VariaProximaRodada'] = dados['TotalGoals_HT'] != dados['TotalGoals_Proxima_Rodada']

    # Filtrar dados conforme a condição estabelecida
    dados_varia = dados[dados[f'VariaUltimas{n}Rodadas']]

    # Contar total de casos onde houve variação nas últimas N rodadas
    total_varia = dados_varia.shape[0]

    # Contar total de casos onde houve variação na próxima rodada após variação nas últimas N
    varia_na_proxima = dados_varia['VariaProximaRodada'].sum()

    # Calcular a probabilidade
    if total_varia > 0:
        probabilidade = varia_na_proxima / total_varia
        return probabilidade, total_varia  # Retorna a probabilidade e a contagem
    else:
        return None, 0

# Calcular a probabilidade para diferentes valores de N
resultados = {}
for n in range(1, 11):
    probabilidade, total_jogos_testados = calcular_probabilidade_variacao(dados, n)
    if probabilidade is not None:
        resultados[n] = (probabilidade, total_jogos_testados)

# Exibir os resultados
for n, (probabilidade, total_jogos_testados) in resultados.items():
    print(f"Probabilidade de variação na próxima rodada após variação nas últimas {n} rodadas: {probabilidade:.2%}")
    print(f"Total de jogos testados para {n} rodadas: {total_jogos_testados}")


Probabilidade de variação na próxima rodada após variação nas últimas 1 rodadas: 64.71%
Total de jogos testados para 1 rodadas: 17
Probabilidade de variação na próxima rodada após variação nas últimas 2 rodadas: 63.64%
Total de jogos testados para 2 rodadas: 11
Probabilidade de variação na próxima rodada após variação nas últimas 3 rodadas: 70.00%
Total de jogos testados para 3 rodadas: 10
Probabilidade de variação na próxima rodada após variação nas últimas 4 rodadas: 75.00%
Total de jogos testados para 4 rodadas: 8
Probabilidade de variação na próxima rodada após variação nas últimas 5 rodadas: 75.00%
Total de jogos testados para 5 rodadas: 8
Probabilidade de variação na próxima rodada após variação nas últimas 6 rodadas: 66.67%
Total de jogos testados para 6 rodadas: 6
Probabilidade de variação na próxima rodada após variação nas últimas 7 rodadas: 66.67%
Total de jogos testados para 7 rodadas: 6
Probabilidade de variação na próxima rodada após variação nas últimas 8 rodadas: 60.00%

Método 1: suponho que TIME MANDANTE IRÁ ENFRENTAR UM TIME VISITANTE,   
time mandante vem de 4 rodadas anteriores com TotalGoals_HT variando   \
TIME VISITANTE  teve TotalGoals_HT  foi igual a rodada anterior
qual a probabilidade de quando eles se enfretarem o TotalGoals_HT variar

In [4]:
import pandas as pd

# Carregar o arquivo fornecido
file_path_atualizado = 'C:\\Users\\msdof\\OneDrive\\Imagens\\Ligas\\BASE DE DADOS GRANDES\\futpython.xlsx'
dados = pd.read_excel(file_path_atualizado)

# Filtrar dados específicos para a liga "Brazil Serie A" e a temporada "2022"
dados = dados[(dados['League'] == 'Brazil Serie A') & (dados['Season'].astype(str).str.contains('2023'))]

# Ordenar dados por data e rodada
dados = dados.sort_values(by=['Date', 'Rodada'])

def calcular_probabilidade_encontro(dados, n):
    # Criar colunas de variação para o time mandante e visitante
    dados['Mandante_Variacao_HT'] = dados.groupby('Home')['TotalGoals_HT'].transform(lambda x: x != x.shift(1))
    dados['Visitante_Variacao_HT'] = dados.groupby('Away')['TotalGoals_HT'].transform(lambda x: x == x.shift(1))

    resultados = []

    for rodada in dados['Rodada'].unique():
        dados_rodada = dados[dados['Rodada'] == rodada]

        for index, jogo in dados_rodada.iterrows():
            home_team = jogo['Home']
            away_team = jogo['Away']

            # Verificar as condições para o time mandante e visitante
            mandante_condicao = dados[(dados['Home'] == home_team) & (dados['Rodada'] < rodada)].tail(n)['Mandante_Variacao_HT'].all()
            visitante_condicao = dados[(dados['Away'] == away_team) & (dados['Rodada'] < rodada)].tail(1)['Visitante_Variacao_HT'].all()

            if mandante_condicao and visitante_condicao:
                # Garantir que existe uma partida na rodada anterior para o time mandante
                jogos_rodada_anterior = dados[(dados['Home'] == home_team) & (dados['Rodada'] == rodada - 1)]
                if not jogos_rodada_anterior.empty:
                    variacao_ht = jogo['TotalGoals_HT'] != jogos_rodada_anterior['TotalGoals_HT'].values[0]
                    resultados.append(variacao_ht)

    # Calcular a probabilidade de variação no HT
    if len(resultados) > 0:
        probabilidade_variacao = sum(resultados) / len(resultados)
    else:
        probabilidade_variacao = None

    return probabilidade_variacao, len(resultados)

# Definir o valor de N (número de rodadas anteriores)
n = 5

# Calcular a probabilidade
probabilidade_variacao, total_jogos_testados = calcular_probabilidade_encontro(dados, n)

# Exibir os resultados
if probabilidade_variacao is not None:
    print(f"Probabilidade de variação no HT quando o time mandante vem de {n} rodadas com variação e o time visitante teve HT igual na rodada anterior: {probabilidade_variacao:.2%}")
    print(f"Total de jogos testados: {total_jogos_testados}")
else:
    print("Não foi possível calcular a probabilidade devido à falta de dados suficientes.")


Probabilidade de variação no HT quando o time mandante vem de 5 rodadas com variação e o time visitante teve HT igual na rodada anterior: 85.71%
Total de jogos testados: 7


In [6]:
import pandas as pd

# Carregar o arquivo fornecido
file_path_atualizado = 'C:\\Users\\msdof\\OneDrive\\Imagens\\Ligas\\BASE DE DADOS GRANDES\\futpython.xlsx'
dados = pd.read_excel(file_path_atualizado)

# Filtrar dados específicos para a liga "Brazil Serie A" e a temporada "2022"
dados = dados[(dados['League'] == 'Brazil Serie A') & (dados['Season'].astype(str).str.contains('2022'))]

# Ordenar dados por data e rodada
dados = dados.sort_values(by=['Date', 'Rodada'])

def calcular_probabilidade_encontro(dados, n):
    # Criar colunas de variação para o time mandante e visitante
    dados['Mandante_Variacao_HT'] = dados.groupby('Home')['TotalGoals_HT'].transform(lambda x: x != x.shift(1))
    dados['Visitante_Variacao_HT'] = dados.groupby('Away')['TotalGoals_HT'].transform(lambda x: x != x.shift(1))

    resultados_variacao_ht = []
    resultados_ft_2_gols = []

    for rodada in dados['Rodada'].unique():
        dados_rodada = dados[dados['Rodada'] == rodada]

        for index, jogo in dados_rodada.iterrows():
            home_team = jogo['Home']
            away_team = jogo['Away']

            # Verificar as condições para o time mandante e visitante
            mandante_condicao = dados[(dados['Home'] == home_team) & (dados['Rodada'] < rodada)].tail(n)['Mandante_Variacao_HT'].all()
            visitante_condicao = dados[(dados['Away'] == away_team) & (dados['Rodada'] < rodada)].tail(n)['Visitante_Variacao_HT'].all()

            if mandante_condicao and visitante_condicao:
                # Garantir que existe uma partida na rodada anterior para o time mandante
                jogos_rodada_anterior = dados[(dados['Home'] == home_team) & (dados['Rodada'] == rodada - 1)]
                if not jogos_rodada_anterior.empty:
                    variacao_ht = jogo['TotalGoals_HT'] != jogos_rodada_anterior['TotalGoals_HT'].values[0]
                    resultados_variacao_ht.append(variacao_ht)

                    # Verificar se a partida terminou com pelo menos 2 gols
                    ft_2_gols = jogo['TotalGoals_FT'] >= 2
                    resultados_ft_2_gols.append(ft_2_gols)

    # Calcular a probabilidade de variação no HT
    if len(resultados_variacao_ht) > 0:
        probabilidade_variacao_ht = sum(resultados_variacao_ht) / len(resultados_variacao_ht)
    else:
        probabilidade_variacao_ht = None

    # Calcular a probabilidade de a partida terminar com pelo menos 2 gols
    if len(resultados_ft_2_gols) > 0:
        probabilidade_ft_2_gols = sum(resultados_ft_2_gols) / len(resultados_ft_2_gols)
    else:
        probabilidade_ft_2_gols = None

    return probabilidade_variacao_ht, probabilidade_ft_2_gols, len(resultados_variacao_ht)

# Definir o valor de N (número de rodadas anteriores)
n = 3

# Calcular a probabilidade
probabilidade_variacao_ht, probabilidade_ft_2_gols, total_jogos_testados = calcular_probabilidade_encontro(dados, n)

# Exibir os resultados
if probabilidade_variacao_ht is not None:
    print(f"Probabilidade de variação no HT quando tanto o time mandante quanto o time visitante vêm de {n} rodadas com variação: {probabilidade_variacao_ht:.2%}")
else:
    print("Não foi possível calcular a probabilidade de variação no HT devido à falta de dados suficientes.")

if probabilidade_ft_2_gols is not None:
    print(f"Probabilidade de a partida terminar com pelo menos 2 gols: {probabilidade_ft_2_gols:.2%}")
else:
    print("Não foi possível calcular a probabilidade de pelo menos 2 gols devido à falta de dados suficientes.")

print(f"Total de jogos testados: {total_jogos_testados}")



Probabilidade de variação no HT quando tanto o time mandante quanto o time visitante vêm de 3 rodadas com variação: 66.67%
Probabilidade de a partida terminar com pelo menos 2 gols: 60.00%
Total de jogos testados: 15


In [9]:
import pandas as pd

# Carregar o arquivo fornecido
file_path_atualizado = 'C:\\Users\\msdof\\OneDrive\\Imagens\\Ligas\\BASE DE DADOS GRANDES\\futpython.xlsx'
dados = pd.read_excel(file_path_atualizado)

# Filtrar dados específicos para a liga "Brazil Serie A" e a temporada "2022"
dados = dados[(dados['League'] == 'Brazil Serie A') & (dados['Season'].astype(str).str.contains('2022'))]

# Ordenar dados por data e rodada
dados = dados.sort_values(by=['Date', 'Rodada'])

def encontrar_jogos_favoritos(dados, n):
    # Criar coluna de variação para o time mandante
    dados['Mandante_Variacao_HT'] = dados.groupby('Home')['TotalGoals_HT'].transform(lambda x: x != x.shift(1))

    jogos_candidatos = []

    for rodada in dados['Rodada'].unique():
        dados_rodada = dados[dados['Rodada'] == rodada]

        for index, jogo in dados_rodada.iterrows():
            home_team = jogo['Home']

            # Verificar se o time mandante era o favorito (odd <= 1.5)
            if jogo['Odd_H_FT'] <= 1.5:
                # Verificar se o time mandante vem de 5 rodadas seguidas com variação de gols no HT
                mandante_condicao = dados[(dados['Home'] == home_team) & (dados['Rodada'] < rodada)].tail(n)['Mandante_Variacao_HT'].all()

                if mandante_condicao:
                    # Verificar se a partida terminou com pelo menos 2 gols
                    if jogo['TotalGoals_FT'] >= 2:
                        jogos_candidatos.append(jogo)

    return jogos_candidatos

# Definir o valor de N (número de rodadas anteriores)
n = 5

# Encontrar os jogos candidatos
jogos_candidatos = encontrar_jogos_favoritos(dados, n)

# Exibir os resultados
print(f"Total de jogos que atendem aos critérios: {len(jogos_candidatos)}")

for jogo in jogos_candidatos:
    print(f"Rodada: {jogo['Rodada']}, Home: {jogo['Home']}, Away: {jogo['Away']}, TotalGoals_FT: {jogo['TotalGoals_FT']}, HomeOdd: {jogo['Odd_H_FT']}")



Total de jogos que atendem aos critérios: 14
Rodada: 1, Home: Palmeiras, Away: Ceará, TotalGoals_FT: 5, HomeOdd: 1.4
Rodada: 2, Home: Corinthians, Away: Avaí, TotalGoals_FT: 3, HomeOdd: 1.37
Rodada: 3, Home: Atlético Mineiro, Away: Coritiba, TotalGoals_FT: 4, HomeOdd: 1.33
Rodada: 5, Home: Atlético Mineiro, Away: América Mineiro, TotalGoals_FT: 3, HomeOdd: 1.47
Rodada: 6, Home: Atlético Mineiro, Away: Atlético GO, TotalGoals_FT: 2, HomeOdd: 1.25
Rodada: 6, Home: São Paulo, Away: Cuiabá, TotalGoals_FT: 3, HomeOdd: 1.45
Rodada: 12, Home: Palmeiras, Away: Atlético GO, TotalGoals_FT: 6, HomeOdd: 1.36
Rodada: 15, Home: Palmeiras, Away: Atlético PR, TotalGoals_FT: 2, HomeOdd: 1.44
Rodada: 19, Home: Palmeiras, Away: Internacional, TotalGoals_FT: 3, HomeOdd: 1.49
Rodada: 21, Home: Palmeiras, Away: Goiás, TotalGoals_FT: 3, HomeOdd: 1.29
Rodada: 24, Home: Internacional, Away: Juventude, TotalGoals_FT: 4, HomeOdd: 1.45
Rodada: 26, Home: Palmeiras, Away: Juventude, TotalGoals_FT: 3, HomeOdd: 1.3
R